In [24]:
import h5py
import numpy as np
import cv2  # For saving masks (optional)
import os

In [25]:
# Step 1: Load the HDF5 file
def load_h5_data(file_path, target_layers=['ILM', 'PR1', 'BM'], num_samples=3):
    with h5py.File(file_path, 'r') as f:
        # Load only the first num_samples images
        images = f['images'][:num_samples]  # Shape: (num_samples, height, width)
        layer_group = f['layers']
        
        # Filter only the target layers we want
        layers = {}
        for layer_name in target_layers:
            if layer_name in layer_group:
                layers[layer_name] = layer_group[layer_name][:num_samples]  # Shape: (num_samples, 768)
            else:
                print(f"Warning: Layer '{layer_name}' not found in the data")
        
        layer_names = list(layers.keys())
    return images, layers, layer_names

In [26]:
images, layers, layer_names = load_h5_data('/home/suraj/Git/SCR-Progression/e2e/Nemours_Jing_RL_Annotated.h5')
#image and layer shapes
print(f"Image shape: {images.shape}")
for layer_name, layer_data in layers.items():
    print(f"Layer '{layer_name}' shape: {layer_data.shape}")

Image shape: (3, 496, 768)
Layer 'ILM' shape: (3, 768)
Layer 'PR1' shape: (3, 768)
Layer 'BM' shape: (3, 768)


In [27]:
# Step 2: Generate segmentation masks - Corrected Layer Ordering
def generate_masks(images, layers, layer_names, height=None, width=768):
    if height is None:
        height = images.shape[1]  # Get height from actual image dimensions
        
    batch_size = images.shape[0]
    
    # Initialize segmentation masks: batch_size x height x width
    # Class 0: Background, Class 1: ILM to BM, Class 2: BM to PR1
    segmentation_masks = np.zeros((batch_size, height, width), dtype=np.uint8)
    
    print(f"Creating masks for {batch_size} images of size {height}x{width}")
    print("Layer order (top to bottom): ILM -> BM -> PR1")
    
    # Process each image in the batch
    for b in range(batch_size):
        print(f"Processing image {b+1}/{batch_size}")
        
        # Get the layer boundaries for this image
        ilm_line = layers['ILM'][b]      # Shape: (768,) - y-coordinates for each x (topmost)
        bm_line = layers['BM'][b]        # Shape: (768,) - y-coordinates for each x (middle)
        pr1_line = layers['PR1'][b]      # Shape: (768,) - y-coordinates for each x (bottommost)
        
        # Process each column (x-coordinate)
        for x in range(width):
            # Get y-coordinates for the three layers at this x position
            ilm_y = ilm_line[x]    # Topmost layer
            bm_y = bm_line[x]      # Middle layer  
            pr1_y = pr1_line[x]    # Bottommost layer
            
            # Skip if any layer has NaN (no annotation in this region)
            if np.isnan(ilm_y) or np.isnan(bm_y) or np.isnan(pr1_y):
                continue
                
            # Convert to integer pixel coordinates
            ilm_y = int(round(ilm_y))
            bm_y = int(round(bm_y))
            pr1_y = int(round(pr1_y))
            
            # Ensure coordinates are within image bounds
            ilm_y = max(0, min(ilm_y, height-1))
            bm_y = max(0, min(bm_y, height-1))
            pr1_y = max(0, min(pr1_y, height-1))
            
            # Ensure proper ordering: ILM (top) < BM (middle) < PR1 (bottom)
            # Sort the three points to ensure correct ordering
            layer_points = sorted([ilm_y, bm_y, pr1_y])
            top_y, mid_y, bottom_y = layer_points
            
            # Fill the regions:
            # Region 1: From top layer to middle layer (Class 1) - ILM to BM region
            if top_y < mid_y:
                segmentation_masks[b, top_y:mid_y, x] = 1
                
            # Region 2: From middle layer to bottom layer (Class 2) - BM to PR1 region
            if mid_y < bottom_y:
                segmentation_masks[b, mid_y:bottom_y, x] = 2
                
            # Everything else remains Class 0 (background)
        
        # Print some statistics for this image
        unique_classes = np.unique(segmentation_masks[b])
        class_counts = [(cls, np.sum(segmentation_masks[b] == cls)) for cls in unique_classes]
        print(f"  Image {b+1} - Classes and pixel counts: {class_counts}")
    
    return segmentation_masks

In [28]:
# Step 3: Post-process masks (optional)
def smooth_masks(masks, kernel_size=3):
    smoothed_masks = masks.copy()
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    for b in range(masks.shape[0]):
        smoothed_masks[b] = cv2.morphologyEx(masks[b], cv2.MORPH_CLOSE, kernel)
    return smoothed_masks

In [29]:
# Step 4: Save or visualize masks
def save_masks(multiclass_masks, output_dir='masks'):
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"Saving {multiclass_masks.shape[0]} sample masks to {output_dir}/")
    
    for b in range(multiclass_masks.shape[0]):
        # Save multi-class mask (scale values for better visibility)
        multiclass_filename = f'{output_dir}/segmentation_mask_sample_{b}.png'
        cv2.imwrite(multiclass_filename, multiclass_masks[b] * 85)  # Scale 0,1,2 -> 0,85,170 for visibility
    
    print("Segmentation mask classes (corrected layer order):")
    print("  Class 0: Background")
    print("  Class 1: ILM to BM region (upper retinal layers)") 
    print("  Class 2: BM to PR1 region (photoreceptor region)")
    print(f"Unique values in masks: {np.unique(multiclass_masks)}")

In [30]:
# Main function
def main(h5_file_path):
    print(f"Loading data from: {h5_file_path}")
    
    # Load data (only 3 samples, only ILM, PR1, BM layers)
    images, layers, layer_names = load_h5_data(h5_file_path)
    
    print(f"Loaded {images.shape[0]} images with shape {images.shape}")
    print(f"Processing layers: {layer_names}")
    
    # Generate masks
    multiclass_masks = generate_masks(images, layers, layer_names)
    
    print(f"Generated segmentation masks with shape: {multiclass_masks.shape}")
    
    # Optional: Smooth masks to reduce noise
    multiclass_masks = smooth_masks(multiclass_masks)
    
    # Save masks
    save_masks(multiclass_masks)
    
    return multiclass_masks

In [31]:
if __name__ == "__main__":
    h5_file_path = '/home/suraj/Git/SCR-Progression/e2e/Nemours_Jing_RL_Annotated.h5'
    multiclass_masks = main(h5_file_path)

Loading data from: /home/suraj/Git/SCR-Progression/e2e/Nemours_Jing_RL_Annotated.h5
Loaded 3 images with shape (3, 496, 768)
Processing layers: ['ILM', 'PR1', 'BM']
Creating masks for 3 images of size 496x768
Layer order (top to bottom): ILM -> BM -> PR1
Processing image 1/3
  Image 1 - Classes and pixel counts: [(0, 286587), (1, 31342), (2, 62999)]
Processing image 2/3
  Image 2 - Classes and pixel counts: [(0, 286470), (1, 31481), (2, 62977)]
Processing image 3/3
  Image 3 - Classes and pixel counts: [(0, 286502), (1, 31424), (2, 63002)]
Generated segmentation masks with shape: (3, 496, 768)
Saving 3 sample masks to masks/
Segmentation mask classes (corrected layer order):
  Class 0: Background
  Class 1: ILM to BM region (upper retinal layers)
  Class 2: BM to PR1 region (photoreceptor region)
Unique values in masks: [0 1 2]


In [32]:
# Test the corrected mask generation
print("Testing corrected mask generation...")

# Generate masks with correct layer ordering
corrected_masks = generate_masks(images, layers, layer_names)

print(f"\nCorrected mask shape: {corrected_masks.shape}")
print(f"Unique values across all masks: {np.unique(corrected_masks)}")

# Check each image individually
for i in range(corrected_masks.shape[0]):
    unique_vals, counts = np.unique(corrected_masks[i], return_counts=True)
    total_pixels = corrected_masks[i].shape[0] * corrected_masks[i].shape[1]
    percentages = (counts / total_pixels) * 100
    print(f"Image {i+1}:")
    for cls, count, pct in zip(unique_vals, counts, percentages):
        region_name = ["Background", "ILM-to-BM", "BM-to-PR1"][cls]
        print(f"  Class {cls} ({region_name}): {count} pixels ({pct:.1f}%)")

# Save the corrected masks
save_masks(corrected_masks)

Testing corrected mask generation...
Creating masks for 3 images of size 496x768
Layer order (top to bottom): ILM -> BM -> PR1
Processing image 1/3
  Image 1 - Classes and pixel counts: [(0, 286587), (1, 31342), (2, 62999)]
Processing image 2/3
  Image 2 - Classes and pixel counts: [(0, 286470), (1, 31481), (2, 62977)]
Processing image 3/3
  Image 3 - Classes and pixel counts: [(0, 286502), (1, 31424), (2, 63002)]

Corrected mask shape: (3, 496, 768)
Unique values across all masks: [0 1 2]
Image 1:
  Class 0 (Background): 286587 pixels (75.2%)
  Class 1 (ILM-to-BM): 31342 pixels (8.2%)
  Class 2 (BM-to-PR1): 62999 pixels (16.5%)
Image 2:
  Class 0 (Background): 286470 pixels (75.2%)
  Class 1 (ILM-to-BM): 31481 pixels (8.3%)
  Class 2 (BM-to-PR1): 62977 pixels (16.5%)
Image 3:
  Class 0 (Background): 286502 pixels (75.2%)
  Class 1 (ILM-to-BM): 31424 pixels (8.2%)
  Class 2 (BM-to-PR1): 63002 pixels (16.5%)
Saving 3 sample masks to masks/
Segmentation mask classes (corrected layer orde